# Pain Assistant
- This notebook is to replicate what we were able to achieve on the AWS console
- Creating through bedrock-agent for more confirugation and control

In [2]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.34


# Test with potential questions

In [3]:
# Create bedrock agent client
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name or 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime", config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

# knowledge-base-app-assistant
kb_id = 'V2W0LT3GZP'

In [6]:

# R and G is to `retrieve_and_generate`
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn,

            # Retrieval Configuration
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results,
                    'overrideSearchType': 'HYBRID' or 'SEMANTIC'
                    }
                }, 

            # Generation Configuration
            'generationConfiguration': {
                    'promptTemplate': {
                        'textPromptTemplate': prompt_template
                    } ,
                    'inferenceConfig': {
                    'textInferenceConfig': {
                        'maxTokens': 512,
                        'temperature': 1.0,
                        'topP': 1.0
                        }
                    }
                },

            # Orchestration Confirugation
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                    }
                } 
            
            } # Knowledge base configuration
        }
    )
    return response


# Printing Generation Results refers to the citations / references from the KB
def print_generation_results(query, response, print_context = False):
    generated_text = response['output']['text']
    print (query)
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


In [7]:
# Assigning the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
filename = 'Pain_Assistant_V1_Oct15.md'

with open(path+filename, 'r') as f:
    prompt_template = f.read()
    

In [8]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

Is it possible to customize the app without logging in?
Many pain management apps do allow some level of customization without logging in or creating an account. Basic customization options like setting units of measurement or allowing notifications are often available prior to account creation. However, more advanced customization features like saving personal data and integrating wearable device data typically require creating a user account.
---------------------------------------------------- 

Am I required to accept the End User License Agreement?
Sorry, I am unable to assist you with this request.
---------------------------------------------------- 

Why isn’t my specific pain condition listed?
Pain conditions can manifest in many different ways across individuals, so it's understandable that some specific pain presentations may not be listed explicitly. However, most chronic pain conditions can be categorized under broader pain categories or pain mechanisms that have been well

# Using bedrock runtime NOT agent

In [8]:

# Bedrock Agent runtime
pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name = region)

# KB ID
kb_id = 'V2W0LT3GZP'


In [9]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [13]:
# Assigning the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
filename = 'Pain_Assistant_V1_Oct15.md'

with open(path+filename, 'r') as f:
    prompt_template = f.read()


In [14]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>
    
    Do not respond if the prompt is not related to pain Psychology.
    Do not respond if the prompt is not related to <contexts>.
    Do not respond if the prompt is related to usage of the app.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

---------------------------------------------------- 

Is it possible to customize the app without logging in?
The prompt does not seem directly related to pain psychology or the provided contexts. As an expert in pain psychology focused on behavioral interventions and coping strategies, I cannot provide details about using or customizing an app. My role is to offer guidance and support on managing chronic pain from a psychological perspective. Please feel free to ask any questions you have related to coping with pain, psychological approaches, or other aspects of pain psychology.
---------------------------------------------------- 

Am I required to accept the End User License Agreement?
I apologize, but this query about accepting an End User License Agreement does not seem related to pain psychology or the provided contexts. As an AI assistant focused on pain psychology, I cannot provide a helpful response to this question. Please feel free to ask me any questions related to managin

In [17]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions3.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>
    
    Do not respond if the prompt is not related to pain Psychology.
    Do not respond if the prompt is not related to <contexts>.
    Do not respond if the prompt is related to usage of the app.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

---------------------------------------------------- 

Will any of my personal information be shared?
I apologize, but I do not have enough context in the information provided to directly answer your question about whether personal information will be shared. As a pain psychologist, my role is to provide guidance on psychological approaches and coping strategies for managing chronic pain. I cannot advise on personal privacy policies or data sharing practices. However, I'd be happy to discuss techniques for coping with pain or pain-related anxiety if that would be helpful.
---------------------------------------------------- 

Do I need to use the same email for both my Google and Facebook accounts to link them in the app?
I apologize, but this query about linking Google and Facebook accounts is not related to my role as a pain psychologist or the provided context on coping strategies and pain management. I am unable to assist with this request. Please let me know if you have any questi

In [18]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions4-Unrelated.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>
    
    Do not respond if the prompt is not related to pain Psychology.
    Do not respond if the prompt is not related to <contexts>.
    Do not respond if the prompt is related to usage of the app.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

---------------------------------------------------- 

What is the weather like today?
This query "What is the weather like today?" does not appear to be related to pain psychology or the given context about coping strategies and managing chronic pain. As an expert on pain psychology, I cannot provide a relevant response to that kind of general weather question. Please feel free to ask me any questions related to pain management, coping with chronic pain, or other psychological aspects of dealing with pain.
---------------------------------------------------- 

How do I start learning a new language?
I should not provide advice about learning a new language, as that is outside the scope of pain psychology and chronic pain management. My role as a pain psychologist is to offer guidance and coping strategies specifically related to managing chronic pain conditions through psychological approaches. If you have any questions about managing or coping with chronic pain, I would be happy to a